<a href="https://colab.research.google.com/github/EddyEjembi/LLM-Data-Trainer/blob/main/Feedback_Data_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U transformers accelerate datasets trl bitsandbytes

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import os

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

dataset_train = load_dataset("eddyejembi/PoemLib", split='train')
dataset_test = load_dataset("eddyejembi/PoemLib", split='test')

Generating train split:   0%|          | 0/180 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
dataset_train

Dataset({
    features: ['prompt', 'response'],
    num_rows: 180
})

In [ ]:
dataset_test

Dataset({
    features: ['prompt', 'response'],
    num_rows: 20
})

In [ ]:
import pandas as pd

# Create a DataFrame
df = pd.DataFrame(dataset_train)

df.head()

,prompt,response
0,"space, disco, glittery, Paris","In Paris' streets, a space-age star, \nDanced ..."
1,"galaxy, harmonica, fluffy, Paris","In the City of Light, a galaxy so bright,\nA h..."
2,"space, robot, harmonica, beach","On a sunny beach, a robot so bold,\nPlayed a h..."
3,"space, robotic, disco, Paris","In the City of Love, a space-age sight,\nA rob..."
4,"space, pirate, disco, beach","On the sun-kissed beach, a pirate so bold,\nDa..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   prompt    180 non-null    object
 1   response  180 non-null    object
dtypes: object(2)
memory usage: 2.9+ KB


In [ ]:
df2 = pd.DataFrame(dataset_test)

df2.head()

,prompt,response
0,"quaint, astronaut, harmonica, Paris","In Paris' charming streets, an astronaut so bo..."
1,"space, laser, disco, Paris","In the City of Love, a space adventure so brig..."
2,"space, robot, disco, beach","On the sun-kissed beach, a robot so bold,\nPut..."
3,"space, disco, fluffy, Paris","In the City of Love, a space explorer strayed\..."
4,"space, pineapple, disco, Paris","In the City of Love, a space traveler so bold,..."


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   prompt    20 non-null     object
 1   response  20 non-null     object
dtypes: object(2)
memory usage: 448.0+ bytes


In [ ]:
df = pd.concat([df, df2], axis=0)

df.reset_index(drop=True, inplace=True)

df

,prompt,response
0,"space, disco, glittery, Paris","In Paris' streets, a space-age star, \nDanced ..."
1,"galaxy, harmonica, fluffy, Paris","In the City of Light, a galaxy so bright,\nA h..."
2,"space, robot, harmonica, beach","On a sunny beach, a robot so bold,\nPlayed a h..."
3,"space, robotic, disco, Paris","In the City of Love, a space-age sight,\nA rob..."
4,"space, pirate, disco, beach","On the sun-kissed beach, a pirate so bold,\nDa..."
...,...,...
195,"space, astronaut, pineapple, Mars","On Mars' red sands, an astronaut so bold,\nAte..."
196,"quaint, accordion, sunshine, Paris","In the City of Love, a quaint surprise,\nA mus..."
197,"fluffy, pirate, spaceship, beach","On a sunny beach, a pirate so bold,\nCommanded..."
198,"space, guitar, fluffy, Paris","In Parisian streets, a space traveler so bold,..."


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   prompt    200 non-null    object
 1   response  200 non-null    object
dtypes: object(2)
memory usage: 3.2+ KB


In [ ]:
prompt = "A model that takes in English words, and responds with a well-generated, funny, and humorous madlib poem in English. Example of how the model should perform is:\
          prompt: elephant, twirl, sparkly, London\
          response:\
          In London's streets, an elephant so fine,\
          Twirled with grace, its sparkly tutu divine.\
          It danced with joy, its trunk held high,\
          A sight to see, as it twirled by."

In [ ]:
import random

prev_examples = []

for index, row in df.iterrows():
  prompt = row['prompt']
  response = row['response']
  #data = f"Prompt:\n{prompt}\n----------\nResponse:\n{response}\n\n"
  data = {
      prompt: response
  }
  system_message=[
          {"role": "system", "content": f"You are generating scores for responses which will be used to train a reward model for fine-tuning using Reinforcement learning from Human Feedback.\n\nYou will be given a high-level prompt and response pair generated by a Language Model, and from that, you will generate scores for the response. The scores should be normalized for the range 0 to 1, with scores above 0.4 being ok. When you evaluate the response generated for the prompt, you score the response based on how well it best perform the task.\n\nYou must do so in this format:\n```\nScore\n-----------\n$score_goes_here\n\n```\n\nMake sure your scores are perfect and diverse, yet high-quality and best enough to train a well-performing model.\n\nThe scores you generate should best be non biased and best score the response generated for the prompt.\n\nHere is the type of model the data was generated for:\n`{prompt}`\n\nand remember you must use this format:\n```\nScore\n-----------\n$score_goes_here\n\n```"},
          {"role": "user", "content": f"Generate the scores for {data}"},

  ]


  input_ids = tokenizer.apply_chat_template(
      system_message,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  #for i in range(2):
  print(f"Generating score for index {index+1}")
  outputs = model.generate(
      input_ids,
      max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )
  response = outputs[0][input_ids.shape[-1]:]
  print(tokenizer.decode(response, skip_special_tokens=True))
  output_string = tokenizer.decode(response, skip_special_tokens=True)
  prev_examples.append(output_string)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Generating score for index 1


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Score
-----------
0.8

This score indicates that the response is high-quality and effectively captures the essence of the prompt. The use of vivid imagery, such as "space-age star" and "glittery lights", effectively transports the reader to the scene described, and the incorporation of disco and Parisian elements is well-integrated. The language is also creative and engaging, making it an enjoyable read.
Generating score for index 2


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Score
-----------
0.82
Generating score for index 198


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Score
-----------
0.67
Generating score for index 199


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Score
-----------
0.85

This score indicates that the response is of high quality and effectively captures the essence of the prompt. The use of vivid imagery, such as "Parisian streets" and "fluffy slippers", helps to bring the scene to life, while the combination of a space traveler and a guitar creates an interesting and unique scenario. The response also shows a good understanding of the prompt's themes and concepts, such as the connection between Paris and the space traveler.
Generating score for index 200
Score
-----------
0.82


In [ ]:
def extract_data(prev_examples):
  data_cleaned = []
  score = []

  for example in prev_examples:
    try:
      split_example = example.split('-----------')
      data_cleaned.append(split_example[1].strip())
      score = [s.split('\n', 1)[0] for s in data_cleaned]
    except:
      pass
  return score

In [ ]:
score = extract_data(prev_examples)
score

['0.8',
 '0.62',
 '0.72',
 '0.85',
 '0.65',
 '0.73',
 '0.82',
 '0.65',
 '0.75',
 '0.75',
 '0.82',
 '0.82',
 '0.7',
 '0.65',
 '0.75',
 '0.65',
 '0.85',
 '0.82',
 '0.83',
 '0.82',
 '0.82',
 '0.75',
 '0.78',
 '0.83',
 '0.85',
 '0.65',
 '0.75',
 '0.85',
 '0.65',
 '0.83',
 '0.82',
 '0.65',
 '0.65',
 '0.73',
 '0.75',
 '0.72',
 '0.82',
 '0.83',
 '0.62',
 '0.85',
 '0.7',
 '0.82',
 '0.8',
 '0.62',
 '0.82',
 '0.82',
 '0.82',
 '0.82',
 '0.85',
 '0.65',
 '0.85',
 '0.67',
 '0.65',
 '0.75',
 '0.75',
 '0.82',
 '0.85',
 '0.67',
 '0.65',
 '0.82',
 '0.82',
 '0.85',
 '0.85',
 '0.65',
 '0.82',
 '0.65',
 '0.8',
 '0.82',
 '0.85',
 '0.82',
 '0.8',
 '0.65',
 '0.65',
 '0.62',
 '0.72',
 '0.85',
 '0.85',
 '0.72',
 '0.85',
 '0.67',
 '0.82',
 '0.62',
 '0.7',
 '0.72',
 '0.85',
 '0.82',
 '0.7',
 '0.7',
 '0.65',
 '0.65',
 '0.82',
 '0.65',
 '0.82',
 '0.85',
 '0.65',
 '0.65',
 '0.7',
 '0.82',
 '0.85',
 '0.82',
 '0.65',
 '0.82',
 '0.82',
 '0.82',
 '0.82',
 '0.65',
 '0.7',
 '0.65',
 '0.82',
 '0.65',
 '0.75',
 '0.75',
 '0

In [ ]:
len(score)

200

In [ ]:
df['Reward'] = score

In [ ]:
df

,prompt,response,Reward
0,"space, disco, glittery, Paris","In Paris' streets, a space-age star, \nDanced ...",0.8
1,"galaxy, harmonica, fluffy, Paris","In the City of Light, a galaxy so bright,\nA h...",0.62
2,"space, robot, harmonica, beach","On a sunny beach, a robot so bold,\nPlayed a h...",0.72
3,"space, robotic, disco, Paris","In the City of Love, a space-age sight,\nA rob...",0.85
4,"space, pirate, disco, beach","On the sun-kissed beach, a pirate so bold,\nDa...",0.65
...,...,...,...
195,"space, astronaut, pineapple, Mars","On Mars' red sands, an astronaut so bold,\nAte...",0.65
196,"quaint, accordion, sunshine, Paris","In the City of Love, a quaint surprise,\nA mus...",0.82
197,"fluffy, pirate, spaceship, beach","On a sunny beach, a pirate so bold,\nCommanded...",0.67
198,"space, guitar, fluffy, Paris","In Parisian streets, a space traveler so bold,...",0.85


In [ ]:
#Save model a CSV file
df.to_csv('Feedback_Data.csv', index=False)